In [1]:
import lsst.daf.butler as dB
import lsst.obs.lsst as oL
import numpy as np
import matplotlib.pyplot as plt
import copy
from multiprocessing import Pool
from multiprocessing import cpu_count
import yaml

# Loading Collection

In [2]:
#genCollection = "u/lgiraldo/bps_ALLCCD_20220701"
genCollection = "u/lgiraldo/bps_ALLCCD_v2_20220804"
butler = dB.Butler("/repo/main", collections=genCollection)

# Amplifiers labeled as bad 

In [3]:
for i in range(189):
    ptc_data = butler.get("ptc", instrument="LSSTCam", detector=i)
    bad_amps = ptc_data.badAmps
    if len(bad_amps)>1:  ##Verify for nan value and another value at least
        print(f"The detector {i} has bad amplifiers {ptc_data.badAmps}") 

The detector 72 has bad amplifiers ['nan', 'C17']
The detector 117 has bad amplifiers ['nan', 'C10']


# Dictionary structure

In [4]:
ampNames = ['C00', 'C01', 'C02','C03','C04','C05','C06', 'C07', 
           'C10', 'C11', 'C12','C13','C14','C15','C16', 'C17']

value = {ampName: '' for ampName in ampNames}
ptc = {ampName: {"gain": '', "rd_noise": '', "a00": '', "turnoff": ''} for ampName in ampNames}
meanvar = {ampName: {"Mean": '', "Var": ''} for ampName in ampNames}
ptc;

## CCD identifiers

In [5]:
camera = butler.get("camera", instrument="LSSTCam")
det_name_to_id = {}
det_id_to_name = {}
det_name_array = []

for det in camera:
    #print(det.getName(), det.getId())
    detName, detId = det.getName(), det.getId()
    if detId < 189:
        det_name_to_id[detName] = detId
        det_id_to_name[detId] = detName
        det_name_array.append(detName)

In [6]:
all_rafts = {detName: copy.deepcopy(value) for detName in det_name_array}
all_rafts_full = {detName: copy.deepcopy(ptc) for detName in det_name_array}
all_rafts_full_meanvar = {detName: copy.deepcopy(meanvar) for detName in det_name_array}

In [7]:
all_rafts_full_meanvar['R01_S02']

{'C00': {'Mean': '', 'Var': ''},
 'C01': {'Mean': '', 'Var': ''},
 'C02': {'Mean': '', 'Var': ''},
 'C03': {'Mean': '', 'Var': ''},
 'C04': {'Mean': '', 'Var': ''},
 'C05': {'Mean': '', 'Var': ''},
 'C06': {'Mean': '', 'Var': ''},
 'C07': {'Mean': '', 'Var': ''},
 'C10': {'Mean': '', 'Var': ''},
 'C11': {'Mean': '', 'Var': ''},
 'C12': {'Mean': '', 'Var': ''},
 'C13': {'Mean': '', 'Var': ''},
 'C14': {'Mean': '', 'Var': ''},
 'C15': {'Mean': '', 'Var': ''},
 'C16': {'Mean': '', 'Var': ''},
 'C17': {'Mean': '', 'Var': ''}}

# Unique file: one for mean-variance and another for gain, read-out noise, $a_{00}$, and turnoff

In [8]:
date = "2022AUG05"  ## the date is used in the file name

for i in range(189):
    ptc_data = butler.get("ptc", instrument="LSSTCam", detector=i)
    if ptc_data is None:
        print (f"Skipping {i} {det_id_to_name[i]} because cpCovs is None")
        continue
    for amp in ampNames:
        all_rafts_full[det_id_to_name[i]][amp]['rd_noise'] = ptc_data.noise[amp]
        all_rafts_full[det_id_to_name[i]][amp]['gain'] = ptc_data.gain[amp]
        all_rafts_full[det_id_to_name[i]][amp]["a00"] = ptc_data.ptcFitPars[amp][0]
        all_rafts_full[det_id_to_name[i]][amp]["turnoff"] = ptc_data.ptcTurnoff[amp]
        
        all_rafts_full_meanvar[det_id_to_name[i]][amp]["Mean"] = ptc_data.rawMeans[amp]
        all_rafts_full_meanvar[det_id_to_name[i]][amp]["Var"] = ptc_data.rawVars[amp]

#Dump data in a yaml file
with open(f'./Data_files/PTC_allvalues_dict_{date}.yaml', 'w') as outfile:
    yaml.dump(all_rafts_full, outfile)
    
with open(f'./Data_files/PTC_meanvar_dict_{date}.yaml', 'w') as outfile:
    yaml.dump(all_rafts_full_meanvar, outfile)  

In [9]:
all_rafts_full['R01_S00']

{'C00': {'gain': 1.6025940789329864,
  'rd_noise': 6.03524920641777,
  'a00': -2.2007323292646248e-06,
  'turnoff': 74918.56120012027},
 'C01': {'gain': 1.593338988138264,
  'rd_noise': 5.937206183957168,
  'a00': -2.2283163081242095e-06,
  'turnoff': 73830.5111707647},
 'C02': {'gain': 1.5907032906430787,
  'rd_noise': 6.0186215031925245,
  'a00': -2.2262356406581675e-06,
  'turnoff': 72504.39066358109},
 'C03': {'gain': 1.5890874977554625,
  'rd_noise': 6.080678947957753,
  'a00': -2.211855659057467e-06,
  'turnoff': 72801.81962359825},
 'C04': {'gain': 1.583271570703637,
  'rd_noise': 6.2061419898764045,
  'a00': -2.0159958259089e-06,
  'turnoff': 73461.1562084773},
 'C05': {'gain': 1.5879812436631653,
  'rd_noise': 6.022350630043325,
  'a00': -2.172276779939199e-06,
  'turnoff': 73368.42399160544},
 'C06': {'gain': 1.5965776808527221,
  'rd_noise': 6.023938275236298,
  'a00': -2.1560282849817983e-06,
  'turnoff': 73243.72346314258},
 'C07': {'gain': 1.6015312381993183,
  'rd_noise'

# Individual files for gain, read-out noise, $a_{00}$, and turnoff

In [10]:
for par in ['rd_noise', 'gain', "a00", "turnoff"]:
    ##Restarting the dictionary
    all_rafts = {detName: copy.deepcopy(value) for detName in det_name_array}
    
    for i in range(189):
        ptc_data = butler.get("ptc", instrument="LSSTCam", detector=i)
        for amp in ampNames:
            if par == 'rd_noise':
                all_rafts[det_id_to_name[i]][amp] = ptc_data.noise[amp]
            if par == 'gain':
                all_rafts[det_id_to_name[i]][amp] = ptc_data.gain[amp]
            if par == 'a00':
                all_rafts[det_id_to_name[i]][amp] = ptc_data.ptcFitPars[amp][0]
            if par == 'turnoff':
                all_rafts[det_id_to_name[i]][amp] = ptc_data.ptcTurnoff[amp]
    
    #Dump data in a yaml file
    with open(f'./Data_files/PTC_{par}_dict_{date}.yaml', 'w') as outfile:
        yaml.dump(all_rafts, outfile)

In [11]:
all_rafts['R01_S01']

{'C00': 75699.33789562658,
 'C01': 72110.11990948125,
 'C02': 72350.83229197445,
 'C03': 70512.11444503214,
 'C04': 69396.2837295275,
 'C05': 69527.53161113187,
 'C06': 71102.83591098123,
 'C07': 73779.76616058405,
 'C10': 73622.76162876893,
 'C11': 70480.05284783359,
 'C12': 70492.61812892648,
 'C13': 70634.68449453358,
 'C14': 69193.76558605884,
 'C15': 68977.51145611893,
 'C16': 69323.59221447469,
 'C17': 71049.84203553203}